# 1) Titolo e obiettivi

Lezione 33: Named Entity Recognition (NER) - Estrarre entità dal testo

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, cos'è NER | 5 min |
| 2 | Teoria: tipi di entità, BIO tagging, approcci | 15 min |
| 3 | Schema mentale: workflow NER | 5 min |
| 4 | Demo: spaCy, regex, confronto | 25 min |
| 5 | Esercizi guidati | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Capire **cos'è NER** e i tipi di entità | Sai distinguere PER, ORG, LOC, DATE? |
| 2 | Usare **spaCy** per NER | Sai estrarre doc.ents? |
| 3 | Usare **regex** per pattern specifici | Sai estrarre email, date, telefoni? |
| 4 | **Scegliere** tra modello ML e regex | Sai quando preferire quale? |
| 5 | **Valutare** le estrazioni | Sai calcolare precision/recall entity-level? |

---

## L'idea centrale: cos'è Named Entity Recognition

```
INPUT (testo):                                OUTPUT (entità):

"Barack Obama met Angela Merkel              [Barack Obama]    → PER (persona)
 in Berlin on July 10, 2015.                 [Angela Merkel]   → PER (persona)
 Contact: barack@example.com"                [Berlin]          → LOC (luogo)
                                             [July 10, 2015]   → DATE (data)
                                             [barack@...]      → EMAIL (regex)
```

**NER = trovare e classificare le entità nominate nel testo.**

---

## Tipi di entità standard

| Sigla | Tipo | Esempi |
|-------|------|--------|
| **PER** | Persona | Mario Rossi, Angela Merkel |
| **ORG** | Organizzazione | Apple, ONU, Juventus |
| **LOC** | Luogo | Monte Bianco, Milano (località) |
| **GPE** | Geopolitico | Italia, Roma (stato/città) |
| **DATE** | Data | 25/12/2024, lunedì |
| **MONEY** | Importo | €100, 1 milione |
| **MISC** | Altro | Linguaggi, nazionalità |

---

## BIO Tagging: come si annotano le entità

```
Frase:    "Barack  Obama  met  Angela  Merkel"
Token:     Barack  Obama  met  Angela  Merkel
Tag:       B-PER   I-PER  O    B-PER   I-PER

B = Begin (primo token dell'entità)
I = Inside (token successivi della stessa entità)
O = Outside (non è un'entità)
```

**Importante per:** training di modelli NER custom.

---

## Approcci NER: quando usare cosa

| Approccio | Pro | Contro | Quando usare |
|-----------|-----|--------|--------------|
| **spaCy/ML** | Generalizza, copre molti tipi | Richiede modello, meno preciso su pattern fissi | Testi generali, entità varie |
| **Regex** | Preciso, veloce, interpretabile | Non generalizza, mantiene | Pattern fissi: email, CF, date |
| **Gazetteer** | Lookup veloce | Lista finita | Aziende note, nomi specifici |
| **Fine-tuning** | Adatta al dominio | Richiede dati annotati | Dominio specializzato |

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| Tokenizzazione | Lezione 30 | Sai cos'è un token? |
| Regex base | Python re module | Sai usare findall? |
| Precision/Recall | Lezione 17 | Sai calcolarle? |

**Cosa useremo:** spaCy (en_core_web_sm), re module, pattern comuni.

# 2) Teoria concettuale
- NER identifica entita' nominate (persone, organizzazioni, luoghi, date).
- Modelli: basati su reti neurali pre-addestrate (es. spaCy) o regole (regex) per pattern semplici.
- Output: per ogni token, un'etichetta di entita' (BIO/IOB) o span con tipo.


## Quando usare modelli pre-addestrati vs regex
- Modello NER: testi generali, copertura ampia ma richiede dipendenze e modello gia' installato.
- Regex: pattern ripetitivi (email, codici, date con formato noto), semplice e leggibile ma limitato.


# 3) Schema mentale / mappa decisionale
1. Definisci quali entita' servono (PER, ORG, LOC, DATE, custom).
2. Scegli: modello pre-addestrato (copertura ampia) o regex/pattern (precisione su formati).
3. Applica NER e ispeziona esempi estratti.
4. Valuta precision/recall se hai label; altrimenti controlla manualmente campioni.
5. Integra in pipeline: pulizia minima, NER, post-processing (dedup, normalizzazione).


# 4) Sezione dimostrativa
- Demo 1: NER con spaCy su frasi di esempio.
- Demo 2: estrazioni regex per pattern noti (email, date, cifre).
- Demo 3: confronto rapido tra output NER e regex.


## Demo 1 - NER con spaCy
Perche': usare un modello pre-addestrato e vedere le entita' riconosciute. Checkpoint: testo non vuoto, lista di entita' estratte.


In [ ]:
# Setup spaCy (richiede modello pre-addestrato en_core_web_sm)
import spacy
import re

nlp = spacy.load('en_core_web_sm')
text = "Barack Obama met Angela Merkel in Berlin on July 10, 2015. Contact him at barack@example.com"

doc = nlp(text)
print("Entita' riconosciute (label, testo, span):")
for ent in doc.ents:
    print(ent.label_, ent.text, (ent.start_char, ent.end_char))
assert len(doc.ents) > 0, "Nessuna entita' estratta"


## Demo 2 - Regex per pattern specifici
Perche': trovare pattern ripetitivi come email o date formattate.


In [ ]:
# Estrazione regex per email e date semplici
emails = re.findall(r"[\w\.]+@[\w\.]+", text)
dates = re.findall(r"\d{1,2}\s\w+\s\d{4}", text)
print("Email trovate:", emails)
print("Date trovate:", dates)


## Demo 3 - Confronto NER vs regex
Perche': capire differenze: NER generalista vs pattern rigidi.


In [ ]:
# Confronto: entita' NER rispetto a regex per email/date
ent_labels = [(ent.label_, ent.text) for ent in doc.ents]
print("NER entita':", ent_labels)
print("Regex email:", emails)
print("Regex date:", dates)


In [ ]:
# ============================================================
# DEMO 6: Valutazione Metriche NER
# ============================================================
# Come calcolare Precision, Recall, F1 per NER

def evaluate_ner(predictions: List[Entity], 
                 ground_truth: List[Entity],
                 strict: bool = True) -> Dict:
    """
    Valuta le performance di un sistema NER.
    
    Args:
        predictions: entità predette
        ground_truth: entità corrette (gold standard)
        strict: se True, richiede match esatto di span E tipo
                se False, richiede solo overlap di span
    
    Returns:
        Dizionario con P, R, F1 per tipo
    """
    # Converti in set di tuple per confronto
    if strict:
        pred_set = {(e.text.lower(), e.type, e.start, e.end) for e in predictions}
        gold_set = {(e.text.lower(), e.type, e.start, e.end) for e in ground_truth}
    else:
        # Match rilassato: solo tipo deve coincidere
        pred_set = {(e.type, e.text.lower()) for e in predictions}
        gold_set = {(e.type, e.text.lower()) for e in ground_truth}
    
    # True Positives: predetti correttamente
    tp = len(pred_set & gold_set)
    
    # False Positives: predetti ma non nel gold
    fp = len(pred_set - gold_set)
    
    # False Negatives: nel gold ma non predetti
    fn = len(gold_set - pred_set)
    
    # Metriche
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Precision': precision,
        'Recall': recall,
        'F1': f1
    }

# Esempio di valutazione
# Ground truth (annotazioni umane)
gold_entities = [
    Entity('Apple', 'ORG', 12, 17),
    Entity('Tim Cook', 'PER', 19, 27),
    Entity('€10 miliardi', 'MONEY', 72, 84),
    Entity('Italia', 'LOC', 103, 109),
    Entity('2025', 'DATE', 120, 124),
    Entity('Milano', 'LOC', 157, 163),
    Entity('25/03/2025', 'DATE', 167, 177),
    Entity('info@apple.com', 'EMAIL', 198, 212),
    Entity('02-1234567', 'PHONE', 215, 225),
]

# Predizioni del nostro sistema (dalla demo precedente)
pred_entities = entities  # Dal demo 5

print("="*60)
print("DEMO 6: VALUTAZIONE NER")
print("="*60)

# Valutazione stretta
strict_metrics = evaluate_ner(pred_entities, gold_entities, strict=True)
print("\n📊 Valutazione STRICT (span + tipo esatti):")
print(f"  True Positives: {strict_metrics['TP']}")
print(f"  False Positives: {strict_metrics['FP']}")
print(f"  False Negatives: {strict_metrics['FN']}")
print(f"  Precision: {strict_metrics['Precision']:.2%}")
print(f"  Recall: {strict_metrics['Recall']:.2%}")
print(f"  F1-Score: {strict_metrics['F1']:.2%}")

# Valutazione rilassata
relaxed_metrics = evaluate_ner(pred_entities, gold_entities, strict=False)
print(f"\n📊 Valutazione RELAXED (solo tipo corretto):")
print(f"  Precision: {relaxed_metrics['Precision']:.2%}")
print(f"  Recall: {relaxed_metrics['Recall']:.2%}")
print(f"  F1-Score: {relaxed_metrics['F1']:.2%}")

# 5) Esercizi svolti (passo-passo)
## Esercizio 33.1 - Applicare NER a un testo custom
Obiettivo: passare un testo libero e stampare entita' riconosciute.


In [ ]:
# Soluzione esercizio 33.1
custom_text = "Apple will open a new office in Milan in September 2024. CEO Tim Cook will attend."
custom_doc = nlp(custom_text)
for ent in custom_doc.ents:
    print(ent.label_, ent.text)
assert len(custom_doc.ents) > 0


## Esercizio 33.2 - Regex per numeri di telefono
Obiettivo: estrarre numeri di telefono con un pattern semplice.


In [ ]:
# Soluzione esercizio 33.2
phone_text = "Call us at +39 02 123456 or +1-202-555-0147 for support."
phones = re.findall(r"\+?\d[\d\s\-]{6,}\d", phone_text)
print("Telefoni trovati:", phones)
assert len(phones) >= 1


# 6) Conclusione operativa
- NER pre-addestrato copre molte entita' generiche; regex e' utile per formati specifici.
- Scegli in base a coverage vs precisione: modelli neurali per contesto, regex per pattern ripetibili.
- Integrare controlli manuali/spot-check se non ci sono label per valutare.

Metodi spiegati: spaCy NER (doc.ents), regex `findall` per pattern, differenze tra approcci.
Errori comuni: usare modello sbagliato per lingua, non gestire encoding del testo, regex troppo permissive che generano falsi positivi.


# 7) Checklist di fine lezione
- [ ] Ho definito quali entita' servono (PER/ORG/LOC/DATE o custom).
- [ ] Ho scelto modello NER o regex in base ai pattern da trovare.
- [ ] Ho verificato su esempi che le entita' siano estratte correttamente.
- [ ] Ho controllato la lingua del modello NER e l'encoding del testo.
- [ ] Ho documentato i pattern regex usati e le limitazioni.

Glossario
- NER: Named Entity Recognition, estrazione di entita'.
- spaCy: libreria NLP con modelli pre-addestrati.
- Regex: espressioni regolari.
- Entita': span di testo etichettato con un tipo.


---

# 5. Conclusione Operativa

## Cosa Abbiamo Imparato

| Concetto | Definizione | Applicazione |
|----------|-------------|--------------|
| **NER** | Identificazione entità nominate | Information Extraction |
| **BIO Tagging** | Schema di annotazione sequenziale | Training modelli NER |
| **Gazetteers** | Dizionari di entità note | Lookup veloce |
| **Rule-based NER** | Pattern regex per entità | Entità strutturate |
| **Feature Engineering** | Caratteristiche token | NER ML-based |

## Quando Usare Quale Approccio

| Tipo Entità | Approccio Consigliato | Perché |
|-------------|----------------------|--------|
| Date, email, numeri | **Regex** | Pattern fissi e prevedibili |
| Nomi aziende note | **Gazetteer** | Lista finita e stabile |
| Nomi persone generici | **ML/DL** | Variabilità alta |
| Dominio specifico | **Fine-tuning** | Lessico specializzato |

## Workflow Operativo

```python
# Template NER production

# 1. Per entità strutturate → Regex
import re
emails = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', text)

# 2. Per entità note → Lookup
known_orgs = {'apple', 'google', 'microsoft'}
found = [w for w in tokens if w.lower() in known_orgs]

# 3. Per NER completo → Librerie
# spaCy (locale, veloce)
import spacy
nlp = spacy.load("it_core_news_lg")
doc = nlp(text)
entities = [(ent.text, ent.label_) for ent in doc.ents]

# HuggingFace (cloud, accurato)
from transformers import pipeline
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
```

## Errori Comuni da Evitare

1. **Regex troppo greedy**: pattern che matchano troppo
2. **Case sensitivity**: dimenticare `.lower()` nei lookup
3. **Overlap non gestito**: entità sovrapposte non risolte
4. **Valutazione token-level**: usare entity-level per NER
5. **Ignorare contesto**: guardare solo il token, non i vicini

---

# 6. Bignami — Scheda di Riferimento Rapido

## Tipi di Entità Standard

| Sigla | Significato | Esempi |
|-------|-------------|--------|
| PER | Persona | Mario Rossi, Angela Merkel |
| ORG | Organizzazione | Apple, ONU, Juventus |
| LOC | Luogo | Milano, Monte Bianco |
| GPE | Entità geopolitica | Italia, Roma (comune) |
| DATE | Data | 25/12/2024, lunedì |
| MONEY | Importo | €100, 1 milione |
| PERCENT | Percentuale | 5%, dieci per cento |

## Schema BIO

```
B-XXX = Begin: primo token entità tipo XXX
I-XXX = Inside: token successivi stessa entità
O     = Outside: non è un'entità
```

## Pattern Regex Comuni

```python
# Email
r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'

# Telefono italiano
r'(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}'

# Codice fiscale
r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b'

# Data italiana
r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b'

# Importo euro
r'€\s?\d+(?:[.,]\d{2})?|
\d+(?:[.,]\d{2})?\s?€'
```

## Metriche NER

```python
# Entity-level evaluation
Precision = TP / (TP + FP)  # Dei predetti, quanti corretti
Recall = TP / (TP + FN)     # Dei gold, quanti trovati
F1 = 2 * P * R / (P + R)    # Media armonica
```

## Checklist Pre-Deploy

```
□ Pattern testati su edge cases
□ Gazetteers aggiornati
□ Overlap resolution implementato
□ Valutazione entity-level (non token)
□ Test su dati reali del dominio
□ Performance accettabile (>80% F1)
```

---
*Fine Lezione 33 — Named Entity Recognition (NER)*

# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-02-10 | Creazione: spaCy NER base |
| 1.1 | 2024-02-18 | Aggiunto regex per email/date |
| 2.0 | 2024-02-22 | Integrato BIO tagging e confronto approcci |
| 2.1 | 2024-02-28 | Refactor con bignami e workflow |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII entità diagram, tipi entità standard table, BIO tagging explained, confronto approcci (ML vs regex vs gazetteer), workflow operativo template, pattern regex comuni, metriche entity-level, checklist pre-deploy |

---

## Note per lo studente

NER è fondamentale per **Information Extraction**:

| Task | Input | Output NER |
|------|-------|------------|
| Knowledge Graph | Articoli | Nodi: persone, aziende |
| Compliance | Contratti | Entità da anonimizzare |
| Customer Service | Ticket | Prodotti, clienti menzionati |
| News Analysis | Articoli | Chi, dove, quando |

**Pipeline tipica:**
1. Preprocessing → 2. NER (spaCy o regex) → 3. Post-processing → 4. Integrazione

**Prossima tappa:** Lesson 34 - Document Intelligence